![1](ATLASOD.gif)

## Trečia pamoka

Šioje pamokoje mes:
* Iš naujo atrasime Higso bozoną!

In [1]:
import ROOT
from tqdm import tqdm
from PlotStyle import setStyle
setStyle()
%jsroot on
ROOT.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.26/06


In [ ]:
# Duomenų failai su dviem fotonais
# Šį kartą panaudosime TChain klasę, kuri leidžia įskaityti daugiau negu vieną failą
# Veikia labai panašiai į TFile/TTree
# Komentaras: paruošti failai jau turi (bent) du fotonus su pT > 35(25) GeV

medis = ROOT.TChain("mini")
nuoroda = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/"
nuoroda = ""
medis.AddFile(nuoroda+"data_A.GamGam.root")
medis.AddFile(nuoroda+"data_B.GamGam.root")
medis.AddFile(nuoroda+"data_C.GamGam.root")
medis.AddFile(nuoroda+"data_D.GamGam.root")

#Priminimas: rasti fotonų duomenų šakas galima taip:
medis.Print()
print(f"Iš viso analizuojame {medis.GetEntries()} įvykių")

In [ ]:
# Pasitelkiant praeitų dviejų pamokų turiniu, parašykime funkciją kuri nuskaito failą
# ir vietoj dviejų *leptonų* išrenka du *fotonus*
def fotonas_praeina_reikalavimus(įvykis, indeksas):
    if (įvykis.photon_isTightID[indeksas]) and \
    (abs(įvykis.photon_eta[indeksas]) < 1.37 or abs(įvykis.photon_eta[indeksas]) > 1.57) and \
    (įvykis.photon_ptcone30[indeksas]/ įvykis.photon_pt[indeksas] < 0.065) and \
    (įvykis.photon_etcone20[indeksas]/įvykis.photon_pt[indeksas] < 0.065):
        return True
    else:
        return False
    
def gauti_masės_histogramą(medis, n_įvykių,
                           intervalai=30, min_masė=100, max_masė=160):   
    masės_histograma = ROOT.TH1D("duomenys", 
                                 "; m_{#gamma#gamma} [GeV];",
                                 intervalai, min_masė, max_masė)

    # Pastorinkime histogramą kad aiškiau matytųsi grafike
    masės_histograma.SetLineWidth(2)
    masės_histograma.SetLineColor(ROOT.kBlack)
    masės_histograma.SetMinimum(10)
    masės_histograma.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
    masės_histograma.GetYaxis().SetTitle("Įvykiai / 2 GeV")
    pirmas_fotonas = ROOT.TLorentzVector()
    antras_fotonas = ROOT.TLorentzVector()

    n_max = min(medis.GetEntries(), n_įvykių)
    # Panašiai, kaip ir pirmoje pamokoje
    n = 0
    with tqdm(total=n_max) as progresas:
        for įvykis in medis:
            
            if n == n_max:
                break
            n += 1
            progresas.update(1)
            # Turi būti lygiai du fotonai
            if įvykis.photon_n != 2:
                continue
            if įvykis.n_jet < 1:
                continue
            if įvykis.jet_pt[0] < 200:
                continue

            if fotonas_praeina_reikalavimus(įvykis, 0) and fotonas_praeina_reikalavimus(įvykis, 1):

                # Gaukite fotonų judesio kiekius ir energijas

                pirmo_pt = įvykis.photon_pt[0]/1000.
                antro_pt = įvykis.photon_pt[1]/1000.
                pirmo_E = įvykis.photon_E[0]/1000.
                antro_E = įvykis.photon_E[1]/1000.
                pirmo_phi = įvykis.photon_phi[0]
                antro_phi = įvykis.photon_phi[1]
                pirmo_eta = įvykis.photon_eta[0]
                antro_eta = įvykis.photon_eta[1]

                pirmas_fotonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
                antras_fotonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

                # Apskaičiuojame sistemos masę
                sistema = pirmas_fotonas + antras_fotonas
                masė = sistema.M()

                if (pirmo_pt/masė < 0.35) or (antro_pt/masė < 0.25):
                    continue

                masės_histograma.Fill(masė)


    return masės_histograma

Jokio rezonanso fone akivaizdžiai nesimato... Kad sumažinti, funkcijoje turėtume įgyvendinkite šiuos reikalavimus fotonams: $p_\mathrm{T} > 25\,\mathrm{GeV}, ir neapimti 1.37<|\eta|<1.57$. Taipogi turime pareikalauti, kad fotonai tenkina identifikacijos reikalavimus `isTightID`. Tai turėtų padidinti santykį tarp tikrų bei triukšmo fotonų, to pasekoje ir tarp signalo ir triukšmo įvykių (kvadratu, nes kiekvienam įvykiui reikia dviejų fotonų!)

Iš pradžių išbandykite su pvz 100 000 duomenų (visų galimų duomenų analizė gali ilgiau užtrukti)

In [ ]:
histograma = gauti_masės_histogramą(medis, 1e9)
histograma.GetEntries()

In [ ]:
# Sukuriame drobę ir dvi paneles
d = ROOT.TCanvas()
d.Draw()
d.SetBottomMargin(0.2)
virš_panelė = ROOT.TPad("viršutinis", "", 0, 0.4, 1, 1)
apat_panelė = ROOT.TPad("apatinis", "", 0, 0, 1, 0.4)
virš_panelė.SetBottomMargin(0)
apat_panelė.SetTopMargin(0)
apat_panelė.SetBottomMargin(0.4)

virš_panelė.Draw()
apat_panelė.Draw()

# Nupiešiame histogramą
virš_panelė.cd()

# Fono glotninimas -- galima išbandyti įvairias funkcijas
fono_funkcija = "pol3" # ax + bx2 + cx3 + dx4
signalo_funkcija = "gaus"

fonas = ROOT.TF1("fonas", fono_funkcija)
fonas_ir_signalas = ROOT.TF1("fonas_ir_signalas", f"{fono_funkcija}+{signalo_funkcija}(4)")
fonas_ir_signalas.SetParameter(4, 120)
#fonas_ir_signalas.SetParLimits(4, 0, 2000)
fonas_ir_signalas.SetParameter(5, 125)
#fonas_ir_signalas.SetParLimits(5, 110, 130)
fonas_ir_signalas.SetParameter(6, 5)
#fonas_ir_signalas.SetParLimits(6, 0, 50)

signalas = ROOT.TF1("signalas", "gaus(4)", 105, 160)


print("Naudojama glotninimo funkcija fonui:", fonas.GetFormula().GetExpFormula())
print("Naudojama glotninimo funkcija fonui ir signalui:", fonas_ir_signalas.GetFormula().GetExpFormula())


fonas.SetLineColor(ROOT.kBlue)
fonas_ir_signalas.SetLineColor(ROOT.kRed)

histograma.Fit("fonas", "E", "E SAME")
fono_histograma = histograma.Clone("fono_histograma")
fono_histograma.Eval(fonas)

# Draw background
for i in range(4):
    fonas_ir_signalas.SetParameter(i, fonas.GetParameter(i))
signalas.SetLineColor(ROOT.kRed)
signalas.Draw("SAME")

histograma.Fit("fonas_ir_signalas", "E+", "E SAME")
fono_ir_signalo_histograma = histograma.Clone("fono_ir_signalo_histograma")
fono_ir_signalo_histograma.Eval(fonas_ir_signalas)

# Nupieškime santykį tarp glotnintos kreivės ir duomenų apatinėje panelėje
apat_panelė.cd()

skirtumas = histograma.Clone("skirtumas")

# Paverčiama funkciją histograma (iš tolydžios į diskretišką)
skirtumas = histograma - fono_histograma

# ----- Paklaidos -----
for i in range(1, histograma.GetNbinsX()):
    skirtumas.SetBinError(i, histograma.GetBinError(i))
# ---------------------

skirtumas.SetLineColor(2)

# ----- Grafiko pagražinimas  ----- 
skirtumas.GetYaxis().SetTitle("Duom. - fonas")
skirtumas.SetLineColor(ROOT.kBlack)
skirtumas.GetXaxis().SetTitleSize(0.14)
skirtumas.GetYaxis().SetTitleSize(0.08)
skirtumas.GetYaxis().SetTitleOffset(0.85)
skirtumas.GetYaxis().SetNdivisions(503, False)
skirtumas.GetYaxis().CenterTitle()
skirtumas.GetXaxis().SetLabelSize(0.12)
skirtumas.GetYaxis().SetLabelSize(0.07)
skirtumas.Draw("PE")
# ---------------------------------

linija = ROOT.TLine(100, 0, 160, 0)
linija.SetLineStyle(7)
linija.SetLineWidth(3)
linija.Draw("SAME")

# Draw background
for i in [4, 5, 6]:
    signalas.SetParameter(i, fonas_ir_signalas.GetParameter(i))
signalas.SetLineColor(ROOT.kOrange)
signalas.Draw("SAME")

d.cd()
d.Update()
d.SaveAs("mgg.png")

Turėtumėte gauti kažką panašaus į šį grafiką: